# CRNN_OCR_Paper

- 참고 사이트
    - [[한글 OCR] CRNN Model 학습](https://velog.io/@shasha/%ED%95%9C%EA%B8%80-OCR-CRNN-Model-%ED%95%99%EC%8A%B5)
    - [딥러닝을 활용한 한글문장 OCR 프로젝트](https://medium.com/@sunwoopark/%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9D%84-%ED%99%9C%EC%9A%A9%ED%95%9C-%ED%95%9C%EA%B8%80%EB%AC%B8%EC%9E%A5-ocr-%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8-hclt-2019-bb9d17622412)
    - [[python] CRNN 한글 모델 학습하기(1) - AIhub 데이터셋](https://velog.io/@apphia39/python-CRNN-%ED%95%9C%EA%B8%80-%EB%AA%A8%EB%8D%B8-%ED%95%99%EC%8A%B5%ED%95%98%EA%B8%B0)
    - [[#04] AI Hub 한국어 글자체 AI 이미지 데이터 전처리](https://cvml.tistory.com/21)
    - [LMDB 형식 데이터셋이란? | 텍스트 인식 데이터셋 포맷](https://mvje.tistory.com/139)

## CRNN 구조
- Convolution Layers : 입력 이미지로부터 feature 시퀀스 추출 ( -> CNNs )
- Recurrent Layers : 각 프레임마다 라벨 예측 ( -> RNNs )
- Transcriptional Layers : 프레임마다의 예측을 최종 라벨 시퀀스로 변경

## CRNN 모델의 backbone model 함수
backbone 역할을 하는 함수: VGG-16 model 구현
CNN Layer + LSTM(RNN) Layer
- CNN Layer
    - 마지막에 있는 2개의 Fully-Connected Layer 층은 CNN층으로 변경
- LSTM(RNN) Layer
    - LSTM 사용 -> gru = False
    - GRU 사용 -> gru = True
    
1. CRNN parameter
- input_shape: (256, 32, 1)
- num_classes: 87 <- 라벨로 사용되는 class 수

2. CRNN model output_shape (train, predict model 각각 존재)
- [Model Architecture]
- CNN(6) + RNN(LSTM)(2) + FC(1) + Activation(Softmax) = x

- model_train
- Model (inputs=[image_input, labels, input_length, label_length], outputs=ctc_loss)
    ```
    >> output_shape: (None, 1)
    >> train model) input parameter가 4개로 늘어나고, output 값이 ctc_loss를 적용한 값이 나온다.
    model_pred
    Model(image_input, x)
    >> Model의 output은 softmax를 통과하고 나옴
    >> output_shape: (None, 62, 87)
    ```

    
3. Model.summary(): 각 model에서 공통으로 가지는 부분 (model_predicton과 동일)
```
Input
(None, 256, 21, 1) - (#samples, width, height, channel)
Conv1_1 * MaxPooling
(None, 128, 16, 64) - 128 * 16 크기의 feature map 64개 생성
Conv2_1 * MaxPooling
(None, 64, 8, 128) - 64 * 8 크기의 feature map 128개 생성
Conv3_1 * Conv3_2 * MaxPooling
(None, 64, 4, 256) - 64 * 4 크기의 feature map 256개 생성
Conv4_1 * BatchNormalization
(None, 64, 4, 512) - 64 * 4 크기의 feature map 512개 생성
Conv5_1 * BatchNormalization * MaxPooling
(None, 64, 2, 512) - 64 * 2 크기의 feature map 512개 생성
Conv6_1
(None, 64, 1, 512) - 64 * 1 크기의 feature map 512개 생성
Reshape
(None, 64, 512) - reshape(-1, 512)로 모양 변경
Bidirectional LSTM * Bidirectional LSTM
(None, 64, 512)
Dense
(None, 64, 87)
Softmax
(None, 64, 87)
```
이 기본 구조는 model_prediction 할 때도 사용된다.  
model_train과의 차이점은, train 시킬 땐 기존 이미지 input에 3개의 input이 더 추가되고, output value가 ctc loss라는 점이다.

# 실제 코드

## file 구조
- data/OCR_Bulk/TL
    - TrainData 중 Labeling data
- data/OCR_Bulk/TS
    - TrainData 중 Source data
- data/OCR_Bulk/VL
    - ValidateData 중 Labeling data
- data/OCR_Bulk/VS
    - Validate 중 Source data
- Paper 데이터 종류
    - O.Form
        - 양식 내용
    - R.Free
        - 자유롭게 적는 것

In [1]:
import pandas as pd
import numpy as np
import os
import json
import pprint

In [2]:
file_list_json = []
path_list = []

TL_path = 'data/OCR_Bulk/TL/P.Paper/'
VL_path = 'data/OCR_Bulk/VL/P.Paper/'

folders = ['O.Form/', 'R.Free/']

def FileList(path, folders):
    
    file_list_json = []
    
    for folder in folders:
        full_path = path + folder
        
        file_list = os.listdir(full_path)
        
        print(full_path, len(file_list))
        
        file_list_json.append(file_list)
        
    return file_list_json
        
TL_list = FileList(TL_path, folders)
VL_list = FileList(VL_path, folders)

data/OCR_Bulk/TL/P.Paper/O.Form/ 38670
data/OCR_Bulk/TL/P.Paper/R.Free/ 31535
data/OCR_Bulk/VL/P.Paper/O.Form/ 4500
data/OCR_Bulk/VL/P.Paper/R.Free/ 4000


In [3]:
def OpenJson(kind, cnt, dataKind):
    
    if kind == 'O':
        n = 0
        
    elif kind == 'R':
        n = 1
    
    if dataKind == 'T':
        path = TL_path + folders[n] + TL_list[n][cnt]
    elif dataKind == 'V':
        path = VL_path + folders[n] + VL_list[n][cnt]
        
        
    with open(path, 'r', encoding='UTF8') as f:
        data = json.load(f)
        return data

In [4]:
data = OpenJson('O', 2, 'T')
data

{'Annotation': {'object_recognition': 0, 'text_language': 0},
 'Dataset': {'category': 0,
  'identifier': 'IMG_OCR_53',
  'label_path': 'HW_OCR/4.Validation/P.Paper/O.Form/',
  'name': '대용량 손글씨 데이터셋',
  'src_path': 'HW_OCR/4.Validation/P.Paper/O.Form/',
  'type': 1},
 'Images': {'acquistion_location': '공공',
  'application_field': '필사본',
  'background': 0,
  'data_captured': '2021.10.05',
  'height': 3503,
  'identifier': 'IMG_OCR_53_4PO_09453',
  'media_type': 0,
  'pen_color': 'black',
  'pen_type': 0,
  'type': 'png',
  'width': 2468,
  'writer_age': 25,
  'writer_sex': 1,
  'written_content': 0},
 'bbox': [{'data': '09453',
   'id': 1,
   'x': [1843, 1843, 2029, 2029],
   'y': [202, 265, 202, 265]},
  {'data': '주', 'id': 2, 'x': [771, 771, 809, 809], 'y': [649, 716, 649, 716]},
  {'data': '갑분싸',
   'id': 3,
   'x': [839, 839, 1013, 1013],
   'y': [645, 727, 645, 727]},
  {'data': '진속저',
   'id': 4,
   'x': [776, 776, 954, 954],
   'y': [836, 935, 836, 935]},
  {'data': '050',
   'id

In [5]:
# former: n = 0
# Free: n = 1

n = 0

with open(TL_path + folders[n] + TL_list[n][0], 'r', encoding='UTF8') as f:
    data = json.load(f)
pprint.pprint(data)

{'Annotation': {'object_recognition': 0, 'text_language': 0},
 'Dataset': {'category': 0,
             'identifier': 'IMG_OCR_53',
             'label_path': 'HW_OCR/4.Validation/P.Paper/O.Form/',
             'name': '대용량 손글씨 데이터셋',
             'src_path': 'HW_OCR/4.Validation/P.Paper/O.Form/',
             'type': 1},
 'Images': {'acquistion_location': '공공',
            'application_field': '필사본',
            'background': 0,
            'data_captured': '2021.10.05',
            'height': 3505,
            'identifier': 'IMG_OCR_53_4PO_09451',
            'media_type': 0,
            'pen_color': 'black',
            'pen_type': 0,
            'type': 'png',
            'width': 2475,
            'writer_age': 25,
            'writer_sex': 1,
            'written_content': 0},
 'bbox': [{'data': '09451',
           'id': 1,
           'x': [1848, 1848, 2019, 2019],
           'y': [199, 275, 199, 275]},
          {'data': '먹방',
           'id': 2,
           'x': [812, 812, 938, 93

In [6]:
print(data.keys())
print()
for key in data.keys():
    print('<{}>'.format(key))
    print(data[key])
    print()

dict_keys(['Annotation', 'Dataset', 'Images', 'bbox'])

<Annotation>
{'object_recognition': 0, 'text_language': 0}

<Dataset>
{'category': 0, 'identifier': 'IMG_OCR_53', 'label_path': 'HW_OCR/4.Validation/P.Paper/O.Form/', 'name': '대용량 손글씨 데이터셋', 'src_path': 'HW_OCR/4.Validation/P.Paper/O.Form/', 'type': 1}

<Images>
{'acquistion_location': '공공', 'application_field': '필사본', 'background': 0, 'data_captured': '2021.10.05', 'height': 3505, 'identifier': 'IMG_OCR_53_4PO_09451', 'media_type': 0, 'pen_color': 'black', 'pen_type': 0, 'type': 'png', 'width': 2475, 'writer_age': 25, 'writer_sex': 1, 'written_content': 0}

<bbox>
[{'data': '09451', 'id': 1, 'x': [1848, 1848, 2019, 2019], 'y': [199, 275, 199, 275]}, {'data': '먹방', 'id': 2, 'x': [812, 812, 938, 938], 'y': [653, 726, 653, 726]}, {'data': '공맹수', 'id': 3, 'x': [717, 717, 897, 897], 'y': [832, 932, 832, 932]}, {'data': '060', 'id': 4, 'x': [1729, 1729, 1821, 1821], 'y': [769, 838, 769, 838]}, {'data': '8978', 'id': 5, 'x': [1847, 1847

In [7]:
n = 1

with open(TL_path + folders[n] + TL_list[n][0], 'r', encoding='UTF8') as f:
    data = json.load(f)
pprint.pprint(data)

{'Annotation': {'object_recognition': 0, 'text_language': 0},
 'Dataset': {'category': 0,
             'identifier': 'IMG_OCR_53',
             'label_path': 'HW_OCR/4.Validation/P.Paper/R.Free/',
             'name': '대용량 손글씨 데이터셋',
             'src_path': 'HW_OCR/4.Validation/P.Paper/R.Free/',
             'type': 1},
 'Images': {'acquistion_location': '자체',
            'application_field': '기타',
            'background': 0,
            'data_captured': '2021.08.26',
            'height': 3497,
            'identifier': 'IMG_OCR_53_4PR_09180',
            'media_type': 0,
            'pen_color': 'red',
            'pen_type': 1,
            'type': 'png',
            'width': 2505,
            'writer_age': 59,
            'writer_sex': 0,
            'written_content': 1},
 'bbox': [{'data': '009180',
           'id': 1,
           'x': [1873, 1873, 2182, 2182],
           'y': [132, 217, 132, 217]},
          {'data': '광주광역시',
           'id': 2,
           'x': [315, 315, 655, 6

In [8]:
print(data.keys())
print()
for key in data.keys():
    print('<{}>'.format(key))
    print(data[key])
    print()

dict_keys(['Annotation', 'Dataset', 'Images', 'bbox'])

<Annotation>
{'object_recognition': 0, 'text_language': 0}

<Dataset>
{'category': 0, 'identifier': 'IMG_OCR_53', 'label_path': 'HW_OCR/4.Validation/P.Paper/R.Free/', 'name': '대용량 손글씨 데이터셋', 'src_path': 'HW_OCR/4.Validation/P.Paper/R.Free/', 'type': 1}

<Images>
{'acquistion_location': '자체', 'application_field': '기타', 'background': 0, 'data_captured': '2021.08.26', 'height': 3497, 'identifier': 'IMG_OCR_53_4PR_09180', 'media_type': 0, 'pen_color': 'red', 'pen_type': 1, 'type': 'png', 'width': 2505, 'writer_age': 59, 'writer_sex': 0, 'written_content': 1}

<bbox>
[{'data': '009180', 'id': 1, 'x': [1873, 1873, 2182, 2182], 'y': [132, 217, 132, 217]}, {'data': '광주광역시', 'id': 2, 'x': [315, 315, 655, 655], 'y': [1000, 1147, 1000, 1147]}, {'data': '강원도', 'id': 3, 'x': [889, 889, 1148, 1148], 'y': [1002, 1133, 1002, 1133]}, {'data': '경기도', 'id': 4, 'x': [1397, 1397, 1647, 1647], 'y': [976, 1116, 976, 1116]}, {'data': '부산광역시', 'id': 5, 'x

## label에서 필요한 정보
- ID
    - Images > identifier
- File ID
    - Images > identifer + '.' + type
    - 두 요소를 .으로 이어 붙어야 한다.
- Bbox
    - bbox > data, id, x, y 순서로 정렬
    - x와 y에 존재하는 숫자는 좌측 하부, 좌측 상부, 우측 하부, 우측 상부 순서로 정렬되어 있음
    - x, y, x+w, y+h 순서로 정리 필요
        - w는 x3-x1
        - h는 y3-y1
- Text
    - bbox > data

In [12]:
def jsonData(data):
    ID = data['Images']['identifier']
    FileID = data['Images']['identifier'] + '.' + data['Images']['type']
    Bbox = []

    for idBbox in range(0, len(data['bbox'])):
        tmp = []

        for ch in ('x', 'y'):
            if ch == 'x':
                dif = (0, 2)
            elif ch == 'y':
                dif = (0, 1)
            for i in dif:
                tmp.append(data['bbox'][idBbox][ch][i])
        Bbox.append(tmp)

    Text = data['bbox'][0]['data']
    
    return ID, FileID, Bbox, Text

In [13]:
data = OpenJson('O', 2, 'T')
data

{'Annotation': {'object_recognition': 0, 'text_language': 0},
 'Dataset': {'category': 0,
  'identifier': 'IMG_OCR_53',
  'label_path': 'HW_OCR/4.Validation/P.Paper/O.Form/',
  'name': '대용량 손글씨 데이터셋',
  'src_path': 'HW_OCR/4.Validation/P.Paper/O.Form/',
  'type': 1},
 'Images': {'acquistion_location': '공공',
  'application_field': '필사본',
  'background': 0,
  'data_captured': '2021.10.05',
  'height': 3503,
  'identifier': 'IMG_OCR_53_4PO_09453',
  'media_type': 0,
  'pen_color': 'black',
  'pen_type': 0,
  'type': 'png',
  'width': 2468,
  'writer_age': 25,
  'writer_sex': 1,
  'written_content': 0},
 'bbox': [{'data': '09453',
   'id': 1,
   'x': [1843, 1843, 2029, 2029],
   'y': [202, 265, 202, 265]},
  {'data': '주', 'id': 2, 'x': [771, 771, 809, 809], 'y': [649, 716, 649, 716]},
  {'data': '갑분싸',
   'id': 3,
   'x': [839, 839, 1013, 1013],
   'y': [645, 727, 645, 727]},
  {'data': '진속저',
   'id': 4,
   'x': [776, 776, 954, 954],
   'y': [836, 935, 836, 935]},
  {'data': '050',
   'id

In [14]:
ID, FileID, Bbox, Text = jsonData(data)

In [16]:
print(ID, FileID, Bbox, Text, sep = '\n')

IMG_OCR_53_4PO_09453
IMG_OCR_53_4PO_09453.png
[[1843, 2029, 202, 265], [771, 809, 649, 716], [839, 1013, 645, 727], [776, 954, 836, 935], [1722, 1813, 764, 827], [1842, 1962, 762, 829], [1990, 2104, 765, 832], [1725, 1817, 858, 927], [1849, 1974, 859, 925], [2007, 2122, 861, 927], [785, 900, 1162, 1246], [1197, 1271, 1202, 1268], [1289, 1386, 1205, 1272], [1408, 1502, 1200, 1274], [1541, 1620, 1205, 1271], [1640, 1740, 1212, 1273], [1763, 1854, 1215, 1274], [422, 605, 1385, 1469], [1003, 1174, 1351, 1433], [1280, 1479, 1364, 1438], [781, 1100, 1528, 1619], [1123, 1270, 1535, 1617], [1281, 1426, 1543, 1617], [1441, 1588, 1543, 1619], [768, 1015, 1634, 1716], [1029, 1174, 1640, 1716], [1201, 1359, 1640, 1714], [1372, 1514, 1642, 1718], [1935, 1998, 1546, 1617], [2082, 2156, 1548, 1607], [352, 447, 2482, 2551], [523, 615, 2482, 2553], [984, 1129, 2487, 2553], [1264, 1341, 2487, 2548], [1368, 1498, 2492, 2553], [334, 448, 2638, 2704], [491, 560, 2645, 2706], [617, 704, 2645, 2704], [1317, 

### json 파일 병합

In [ ]:
import json
import os
import glob

# JSON 파일 병합 함수
def merge_json_files(input_dir, output_file):
    merged_data = []
    json_pattern = os.path.join(input_dir, '*.json')
    file_list = glob.glob(json_pattern)
    
    for file in file_list:
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            merged_data.append(data)
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(merged_data, f, indent=4)

# 입력 디렉토리와 출력 파일 경로 설정
input_directory = 'path/to/your/json/files'
output_file = 'merged_labels.json'

# JSON 파일 병합 실행
merge_json_files(input_directory, output_file)
print(f"Merged JSON data written to '{output_file}'")


# 모델링

In [17]:
import tensorflow as tf
import numpy as np
import cv2
import json
import matplotlib.pyplot as plt

In [ ]:
# JSON 파일 로드 함수
file_list_json = []
path_list = []

TL_path = 'data/OCR_Bulk/TL/P.Paper/'
VL_path = 'data/OCR_Bulk/VL/P.Paper/'

folders = ['O.Form/', 'R.Free/']

def FileList(path, folders):
    
    file_list_json = []
    
    for folder in folders:
        full_path = path + folder
        
        file_list = os.listdir(full_path)
        
        print(full_path, len(file_list))
        
        file_list_json.append(file_list)
        
    return file_list_json
        
TL_list = FileList(TL_path, folders)
VL_list = FileList(VL_path, folders)

def OpenJson(kind, cnt, dataKind):
    
    if kind == 'O':
        n = 0
        
    elif kind == 'R':
        n = 1
    
    if dataKind == 'T':
        path = TL_path + folders[n] + TL_list[n][cnt]
    elif dataKind == 'V':
        path = VL_path + folders[n] + VL_list[n][cnt]
        
        
    with open(path, 'r', encoding='UTF8') as f:
        data = json.load(f)
        return data

# 이미지 전처리 함수
def preprocess_image(image_path, bounding_box):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    x, y, w, h = bounding_box
    cropped_image = image[y:y+h, x:x+w]
    resized_image = cv2.resize(cropped_image, (128, 32))  # CRNN 입력 크기에 맞게 조정
    normalized_image = resized_image / 255.0
    return normalized_image

# JSON 파일 경로
data = []
for i, item in enumerate(TL_list):
    data.append(OpenJson('O', i, 'T'))

# 이미지 경로와 바운딩 박스 정보
image_path = data['file_name']
bounding_box = data['bounding_box']

# 이미지 전처리
processed_image = preprocess_image(image_path, bounding_box)

In [ ]:
# CRNN 모델 정의 (간단한 예제)
def build_crnn_model(input_shape, num_classes):
    input_layer = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Reshape((-1, x.shape[-1]))(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(x)
    x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.models.Model(inputs=input_layer, outputs=x)
    return model

# 모델 입력 크기와 클래스 수 정의
input_shape = (32, 128, 1)
num_classes = 37  # 예: 알파벳 26자 + 숫자 10자 + 공백 1자

# 모델 빌드
model = build_crnn_model(input_shape, num_classes)

# 모델 요약 출력
model.summary()

# 예측 수행
processed_image = np.expand_dims(processed_image, axis=-1)  # 채널 차원 추가
processed_image = np.expand_dims(processed_image, axis=0)  # 배치 차원 추가
predictions = model.predict(processed_image)

# 예측 결과 출력 (예: 가장 높은 확률의 클래스 인덱스)
predicted_class = np.argmax(predictions, axis=-1)
print(f"Predicted class: {predicted_class}")
